# Zillow House Prices

Definitions used for this project:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [126]:
# Dictionary to map US state names to two-letter acronysms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [127]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

'''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''


### List of University towns in the US

In [128]:
data = pd.read_csv("university_towns.txt",sep= '\n',names = ['RegionName'])
data.insert(0,'State',data['RegionName'].str.extract('(.*)\[edit\]',expand = False).ffill())
data['RegionName'] = data['RegionName'].str.split('\[').str[0]
data['RegionName'] = data['RegionName'].str.split('\s\(').str[0]
unitowns = data[data['State'] != data['RegionName']]
unitowns

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


### GDP dataset and returning the zipped recession period

In [129]:
'''Return the GDP data command and zipped recession start and end'''
gdp = (pd.read_excel("gdplev.xls",skiprows = 7, nrows = 279,usecols = [4,5,6], names = ['Quarter',
                'GDPQ billions current dollars','GDPQ billions 2009 dollars']))
gdp['GDPchange'] = gdp['GDPQ billions 2009 dollars'].diff()

# Counting the number of consecutive quarters with GDP decline and the beginnning quarter
cntdecline = pd.DataFrame(columns = ['Count','Beg_Quarter']).fillna(0)
j = 0
a = 0
for i in range(1,len(gdp) - 1):
    i += a
    if(gdp.loc[i,'GDPchange'] < 0) & ((gdp.loc[i-1,'GDPchange'] > 0)):
        a = 0
        cntdecline.loc[j,'Count'] = a
        cntdecline.loc[j,'Beg_Quarter'] = gdp.loc[i,'Quarter']
        while ((gdp.loc[i,'GDPchange'] < 0) & (i < 278)):
            cntdecline.loc[j,'Count'] += 1
            a += 1
            i += 1
        j += 1        
    else :
        a = 1

# Counting the number of consecutive quarters with GDP rise and the second quarter
cntrise = pd.DataFrame(columns = ['Count','Beg_Quarter'])
st = 0
k = 0    
for i in range(1,len(gdp) - 1):
    i += st
    if (i >= 278):
        continue
    if((gdp.loc[i,'GDPchange'] > 0)  & (gdp.loc[i-1,'GDPchange'] < 0)):
        #print(i)
        st = 0
        cntrise.loc[k,'Count'] = a
        c = i
        while ((i < 278) & (gdp.loc[i,'GDPchange'] > 0)):
            cntrise.loc[k,'Beg_Quarter'] = gdp.loc[c+1,'Quarter']
            cntrise.loc[k,'Count'] += 1
            st += 1
            i += 1
            if (i >= 278):
                break
        k += 1        
    else :
        st = 1
cntrise = cntrise.drop(0).reset_index(drop = True)
  
# Merging the beginning and end of recessions
recesp =cntdecline.merge(cntrise,how= 'inner',left_index = True, right_index = True,suffixes =('d','r'))
gdp,recesp

(    Quarter  GDPQ billions current dollars  GDPQ billions 2009 dollars  \
 0    1947q1                          243.1                      1934.5   
 1    1947q2                          246.3                      1932.3   
 2    1947q3                          250.1                      1930.3   
 3    1947q4                          260.3                      1960.7   
 4    1948q1                          266.2                      1989.5   
 5    1948q2                          272.9                      2021.9   
 6    1948q3                          279.5                      2033.2   
 7    1948q4                          280.7                      2035.3   
 8    1949q1                          275.4                      2007.5   
 9    1949q2                          271.7                      2000.8   
 10   1949q3                          273.3                      2022.8   
 11   1949q4                          271.0                      2004.7   
 12   1950q1             

### Start of the Recession 

In [130]:
'''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
GDP,Rec = gdp,recesp
start = Rec.loc[Rec['Countd']==Rec['Countd'].max()]['Beg_Quarterd'].to_string(header = False, index = False)
start

'2008q3'

### End of the Recession

In [131]:
'''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
GDP, Rec = gdp,recesp
end = Rec.loc[Rec['Countd']==Rec['Countd'].max()]['Beg_Quarterr'].to_string(header = False, index = False)
end

'2009q4'

### Bottom of the Recession

In [132]:
'''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
GDP,Rec = gdp,recesp
start = Rec.loc[Rec['Countd']==Rec['Countd'].max(),'Beg_Quarterd'].to_string(header = False, index = False)
n = Rec['Countd'].max()
bottom = GDP.loc[GDP[GDP['Quarter'] == start].index.values + (n-1),'Quarter'].to_string(header = False, index = False)
bottom

'2009q2'

### Reading and converting Zillow House prices data to Quarters 

In [133]:
'''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
house = pd.read_csv("City_Zhvi_AllHomes.csv",header = 0)
#t = house.columns[house.columns.str.contains('\d\d\d\d')]  # Index containing all date column names from original data
#dtInd = pd.date_range(t.values[0],periods = 245,freq = 'M') # forming dateIndex from t
    
# Creating a subset of house data with all the columns with date labels
house1 = house.iloc[:,6:].T
house1.index = pd.to_datetime(house1.index)  # Converting the index to datetimeIndex
    
# Sampling all the date column labels as a quarter and from 2001 onward
mn_pric = house1[house1.index > '2000-12-31'].resample('Q').mean()
    
#merging back the quartered data with the original data
data1 = house.iloc[:,0:5].merge(mn_pric.T, how = 'inner', left_index = True, right_index = True)
data1 = data1.reset_index()
#data1 = data1.set_index(['State','RegionName'])

#Creating another subset of merged data to convert column date labels to the format 'yyyyq#'
data2 = data1.iloc[:,6:].T
data2.index = pd.to_datetime(data2.index)
data2.index = data2.index.year.astype('str') + 'q' + data2.index.quarter.astype('str')
houseprices = data1.iloc[:,0:6].merge(data2.T.iloc[:,:], how = 'inner',left_index = True, right_index = True)
houseprices = houseprices.set_index('State')
    
# Using the 'states' dictionary to replace two letter acronym with state names
for key,values in states.items():
    houseprices.index.values[houseprices.index == key] = values
houseprices = houseprices.reset_index()
houseprices = houseprices.set_index(['State','RegionName'])
houseprices.sort_index(level =0,inplace = True)
houseprices

index  RegionID                Metro   CountyName  \
State     RegionName                                                           
Alabama   Adamsville        5344     10040           Birmingham    Jefferson   
          Alabaster         1680     43839           Birmingham       Shelby   
          Albertville       1561      8332          Albertville     Marshall   
          Arab              3087     35038          Albertville     Marshall   
          Ardmore           5433     57107           Huntsville      Madison   
          Axis              8799     23564               Mobile       Mobile   
          Baileyton         9914     44007              Cullman      Cullman   
          Bay Minette       3030      3562               Daphne      Baldwin   
          Bayou La Batre    9801     10284               Mobile       Mobile   
          Bessemer          2125     16997           Birmingham    Jefferson   
          Birmingham         120     10417           Birmingham    Jefferson   
          Boaz              5093      3732          Albertville     Marshall   
          Brent             7767      3798           Birmingham         Bibb   
          Brighton          7865     37574           Birmingham    Jefferson   
          Brookwood         6497      3842           Tuscaloosa   Tuscaloosa   
          Buhl              9866     17223           Tuscaloosa   Tuscaloosa   
          Calera            3509     44402           Birmingham       Shelby   
          Center Point      4543     44519           Birmingham    Jefferson   
          Centreville       6328     44535           Birmingham         Bibb   
          Chalkville        9020    103474           Birmingham    Jefferson   
          Chancellor        9285      4043           Enterprise       Coffee   
          Chelsea          10035     37845           Birmingham       Shelby   
          Chickasaw         5025     30870               Mobile       Mobile   
          Chunchula         7949      8211               Mobile       Mobile   
          Citronelle        4416     48822               Mobile       Mobile   
          Clay              9837     57092           Birmingham    Jefferson   
          Coden             9557     24145               Mobile       Mobile   
          Coker             9144     44672           Tuscaloosa   Tuscaloosa   
          Concord           9948     37993           Birmingham    Jefferson   
          Cottondale        3882     36819           Tuscaloosa   Tuscaloosa   
...                          ...       ...                  ...          ...   
Wisconsin Vernon            6232    398534            Milwaukee     Waukesha   
          Vienna            8354    398543              Madison         Dane   
          Vinland           7989    396938              Oshkosh    Winnebago   
          Wales            10357     48303            Milwaukee     Waukesha   
          Waterford         6456    399085               Racine       Racine   
          Waukesha           819     34683            Milwaukee     Waukesha   
          Waunakee          7609     34685              Madison         Dane   
          Waupun            2963     55059          Fond du Lac  Fond du Lac   
          Wausau            1653     14501               Wausau     Marathon   
          Wayne             8087    397462            Milwaukee   Washington   
          West Allis         654     34721            Milwaukee    Milwaukee   
          Weston            4209     21260               Wausau     Marathon   
          Wheatland         6743    398205              Chicago      Kenosha   
          Whitelaw          8933     29264            Manitowoc    Manitowoc   
          Williams Bay      9097     41713           Whitewater     Walworth   
          Wilson            7955    398518            Sheboygan    Sheboygan   
          Wilson            9821     55222  Minneapolis-St Paul  Saint Croix   
          Wind Lake        

### Returning the t-test decision on hypothesis and the correspondng p-value

In [134]:
'''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
# Obtaining list of university towns, recession start, recession bottom, and house pricing data
unitowns = unitowns.set_index(['State','RegionName'])
    
# Obtaining column poistion so as to extract prices before recession start
a = houseprices.columns.get_loc(start)
b = houseprices.columns.get_loc(bottom)
    
# Extracting house prices before recession start and at recession bottom and calculating the price growth/decline
testdata = houseprices.iloc[:,[a-1,b]]
testdata.loc[:,'priceratio'] = testdata.iloc[:,0]/testdata.iloc[:,1]
    
# Separating the testdata into two groups: university towns and non-university towns
idx = pd.IndexSlice
UTdata = pd.DataFrame(columns = ['priceratio'])
NonUTdata = pd.DataFrame(columns = ['priceratio'])

merged = unitowns.merge(testdata,how ='outer', left_index = True, right_index = True, indicator = True)
UTdata = merged.loc[merged['_merge'] == 'both',:]
NonUTdata =  merged.loc[merged['_merge'] == 'right_only',:]

from scipy import stats
tt = stats.ttest_ind(UTdata['priceratio'],NonUTdata['priceratio'],nan_policy = 'omit')
different = False
if (tt.pvalue < 0.01):
    different = True
UTmeanpriceratio = UTdata.loc[:,'priceratio'].mean()
NonUTmeanpriceratio = NonUTdata.loc[:,'priceratio'].mean()
better = 'university town'
if (UTmeanpriceratio > NonUTmeanpriceratio):
    better = 'non-university town'
different, tt.pvalue, better

(True, 0.002724063704753125, 'university town')